<a href="https://colab.research.google.com/github/vbaidalov/Exemple/blob/main/%D0%9A%D0%BE%D0%BF%D0%B8%D1%8F_%D0%B1%D0%BB%D0%BE%D0%BA%D0%BD%D0%BE%D1%82%D0%B0_%22Untitled1_ipynb%22.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd

In [ ]:
# @title Работаем с файлом данных за 2023 год
df = pd.read_csv('/content/drive/MyDrive/База обработка Инсталл 2023/df_sales_2023_clean.csv')
df['Дата реализации'] = pd.to_datetime(df['Дата реализации'], dayfirst=True)
# Преобразуем отсрочку в число
df['Отсрочка'] = pd.to_numeric(df['Отсрочка'], errors = 'coerce')
df['Отсрочка'].fillna(0, inplace = True)

In [ ]:
# @title просто посмотреть структуру файла:
df.head()

In [ ]:
# @title Товары Владимира Башмачникова, чемпионы по выручке
df[['Товар', 'Количество', "Выручка", 'Наценка, %']]\
.loc[df['Продакт']=='Башмачников Владимир Константинович']\
.groupby(by = 'Товар').agg({'Количество': 'sum', 'Выручка':'sum', 'Наценка, %':'mean'})\
.sort_values(by = 'Выручка', ascending = False)\
.head(10)

In [ ]:
# @title Товары Байдалов Василий Игоревич, чемпионы по выручке
df[['Товар', "Артикул", 'Количество', "Выручка", 'Наценка, %']]\
.loc[df['Продакт']=='Байдалов Василий Игоревич']\
.groupby(by = 'Товар')\
.agg({"Артикул":'first','Количество': 'sum', 'Выручка':'sum', 'Наценка, %':'mean'})\
.sort_values(by = 'Выручка', ascending = False)\
.head(10)

In [ ]:
# @title Смотрим в разрезе конкретного товара ONU(GPON-GEPON) PX001. Кому он нужен, какая наценка, сколько берут?
df[['Контрагент', "Филиал",'Количество', 'Выручка', 'Наценка, %']]\
.loc[df['Товар']=='Абонентский терминал V-SOL V2801RD ONU(GPON-GEPON) GE']\
.groupby(by = 'Контрагент')\
.agg({"Филиал":'first','Количество':'sum',  'Выручка':'sum', 'Наценка, %':'mean'})\
.sort_values(by = 'Выручка', ascending = False)\
.head(10)

In [ ]:
# @title Смотрим в разрезе конкретного артикула PX001. Кому он нужен, какая наценка, сколько берут?
df[['Контрагент', "Филиал",'Количество', 'Выручка', 'Наценка, %']]\
.loc[df['Артикул']=='PX001']\
.groupby(by = 'Контрагент')\
.agg({"Филиал":'first','Количество':'sum',  'Выручка':'sum', 'Наценка, %':'mean'})\
.sort_values(by = 'Выручка', ascending = False)\
.head(10)

In [ ]:
# @title Несложная моделька. Дает рейтинг провайдеров в филиале РнД по MEDIAN наценке
pd.pivot_table(df[['ИНН','Контрагент', "Менеджер",'Количество', 'Выручка', 'Себестоимость', 'Валовая прибыль', 'Наценка, %']]\
.loc[(df['Тип компании']=='6. ПРОВАЙДЕР') & (df['Филиал'] == 'Ростов-на-Дону')],\
               values= ["Количество",'Выручка', 'Себестоимость', "Валовая прибыль", 'Наценка, %'], \
               index=['ИНН', 'Контрагент', 'Менеджер'],\
               aggfunc = {"Количество": 'sum',
                          'Выручка':'sum',
                          'Себестоимость':'sum',
                          "Валовая прибыль":'sum',
                          'Наценка, %':'median'})\
               .sort_values(by='Наценка, %', ascending=False)

In [ ]:
# @title Посмотрим на портфель продуктов интересного ростовского провайдера (ранжир по выручке). Ой, а это точно провайдер?
df[['Товар', 'Количество', 'Выручка', 'Валовая прибыль', 'Себестоимость', 'Наценка, %']]\
.loc[df['Контрагент'] =='ВИСТ ОН-ЛАЙН ЗАО']\
.groupby(by = 'Товар')\
.agg({'Наценка, %': 'mean',
      'Количество': 'sum',
      'Выручка':'sum',
      'Валовая прибыль':'sum',
      'Себестоимость': 'sum'})\
.sort_values(by = 'Выручка', ascending = False)

In [ ]:
# @title Хорошо, смотрим на портфель другого ростовского провайдера (ранжир по выручке). Ой...
df[['Товар', 'Количество', 'Выручка', 'Валовая прибыль', 'Себестоимость', 'Наценка, %']]\
.loc[df['Контрагент'] =='ЭЛЛКО ООО']\
.groupby(by = 'Товар')\
.agg({'Наценка, %': 'mean',
      'Количество': 'sum',
      'Выручка':'sum',
      'Валовая прибыль':'sum',
      'Себестоимость': 'sum'})\
.sort_values(by = 'Выручка', ascending = False)

In [ ]:
# @title а здесь еще и фамилию менеджера посмотрим... (ранжир по Кол-ву)
df[['Товар', 'Количество', 'Выручка', 'Валовая прибыль', 'Себестоимость', 'Наценка, %', 'Менеджер']]\
.loc[(df['ИНН'] ==2723074008) & (df['Себестоимость']>1.2)]\
.groupby(by = 'Товар')\
.agg({'Наценка, %': 'mean',
      'Количество': 'sum',
      'Выручка':'sum',
      'Валовая прибыль':'sum',
      'Себестоимость': 'sum',
      'Менеджер':'first'})\
.sort_values(by = 'Количество', ascending = False)